In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt 
import numpy as np
import torch
from itertools import product

import nmf.mult
import nmf.pgrad
import nmf.nesterov

import nmf_torch.mult
import nmf_torch.pgrad
import nmf_torch.nesterov
import nmf_torch.norms

from performance.performance_eval_func import get_random_lowrank_matrix, get_time_ratio,\
                                              compare_performance, plot_performance_dict,\
                                              torch_algo_wrapper,\
                                              plot_ratios_gpu_algo, plot_ratios_cpu_gpu, plot_ratios_cpu_algo
                                                

In [5]:
def time_for_n_steps_over_inner_dim_data_size(r_list, shape_list, n, algo): 
    time_data = np.zeros((len(r_list), len(shape_list))) 
    
    for r_i, r in enumerate(r_list):
        for sh_i, shape in enumerate(shape_list):
            V = get_random_lowrank_matrix(shape[0], r, shape[1])
            W_init, H_init = nmf.mult.update_empty_initials(V, r, None, None)
            W, H, errors = algo(V=V, inner_dim=r, 
                        record_errors=True,
                        time_limit=np.inf,
                        max_steps=n,
                        epsilon=0,
                        W_init=W_init.copy(),
                        H_init=H_init.copy()) 
            time_data[r_i, sh_i] = errors[n-1, 0]

    return time_data

In [6]:
algo_dict_to_test = {
    "mult": nmf.mult.factorise_Fnorm,
    "pgrad": nmf.pgrad.factorise_Fnorm_subproblems,
    "nesterov": nmf.nesterov.factorise_Fnorm,

    "mult_torch": torch_algo_wrapper(nmf_torch.mult.factorise_Fnorm, 
                                     device="cuda"),
    "pgrad_torch": torch_algo_wrapper(nmf_torch.pgrad.factorise_Fnorm_subproblems, 
                                      device="cuda"),
    "nesterov_torch": torch_algo_wrapper(nmf_torch.nesterov.factorise_Fnorm, 
                                        device="cuda")
}

In [6]:
errors_result = errors_at_time_t_over_inner_dim(V, [20, 25, 30, 35, 40, 45,
                                                    50, 
                                                    55, 60, 65, 70, 75, 80, 85, 90],
                                                60, algo_dict_to_test)

NameError: name 'errors_at_time_t_over_inner_dim' is not defined

In [ ]:
f, ax = plt.subplots()
plot_dict(errors_result, ax, log=False)
f, ax = plt.subplots()
plot_dict(errors_result, ax, log=True)

In [7]:
shapes = [(5 * a, a) for a in [30, 100, 300, 1000
                               #, 3000
                              ]]
shapes

[(150, 30), (500, 100), (1500, 300), (5000, 1000)]

In [8]:
inner_dims_small = [sh[1] // 10 for sh in shapes]
inner_dims_small

[3, 10, 30, 100]

In [9]:
inner_dims_big = [8 * sh[1] // 10 for sh in shapes]
inner_dims_big

[24, 80, 240, 800]

In [10]:
shapes_all = shapes + shapes
inner_dims = inner_dims_small + inner_dims_big

In [11]:
times = [5, 25, 125, 725] # + [5, 25, 125, 725]

In [ ]:
errors_dict = {}

In [ ]:
for inner_dim, shape, t  in zip(inner_dims, shapes_all, times):
    if t != 725:
        continue
    
    print(inner_dim, shape)
    V = get_random_lowrank_matrix(shape[0], inner_dim, shape[1]) + np.random.rand(*shape) * 0.1

    W_init = np.random.rand(shape[0], inner_dim)
    H_init = np.random.rand(inner_dim, shape[1])

    errors = compare_performance(V=V, inner_dim=inner_dim, time_limit=t,
                                      W_init=W_init, H_init=H_init, 
                                      algo_dict_to_test=algo_dict_to_test)
    errors_dict[(inner_dim, shape)] = errros
    
    f, axs = plt.subplots(2, 2)
    f.suptitle("Factorising random data size {} with inner dim={}".format(shape, inner_dim))
    axs[0, 0].set_title("log(error)")

    plot_performance_dict(errors, axs[0, 0])
    plot_ratios_cpu_gpu(errors, axs[0, 1])
    plot_ratios_cpu_algo(errors, axs[1, 0])
    plot_ratios_gpu_algo(errors, axs[1, 1])
    
    break

100 (5000, 1000)


In [13]:
errors_dict

{}

In [ ]:
def plot_comparison(errors_dict, ax_grid):
    plot_performance_dict(errors_dict, ax_grid[0, 0])
    ax_grid[0, 0].xlabel("time[s]")
    ax_grid[0, 0].ylabel("log(error)")
    
    plot_ratios_cpu_gpu(errors_dict, ax_grid[0, 1])
    plot_ratios_cpu_algo(errors_dict, ax_grid[1, 0])
    plot_ratios_gpu_algo(errors_dict, ax_grid[1, 1])
   